In [ ]:
#! pip install chardet


import folium 
import chardet


import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 


from pandas import Series, DataFrame
from folium import Marker, GeoJson
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
from folium.plugins import MarkerCluster, HeatMap

# Data reading and cleaning

In [ ]:
base = open(r'../../_data/data_dengue_peru.csv','rb').read()
det = chardet.detect(base)
charenc = det['encoding']

# Reading data
df = pd.read_csv(r'../../_data/data_dengue_peru.csv', encoding = charenc)

# Generating Ubigeo as string, cause not all Ubigeo values are of 6 digits
df["Ubigeo_str"]  = df['Ubigeo'].astype(str).str.zfill(6)

# Identifying department
df["Ubigeo_dep"]  = df["Ubigeo_str"].str[0:2].astype(int)

# Identifying province
df["Ubigeo_prov"] = df["Ubigeo_str"].str[0:4].astype(int)

# Capital letter to merge
df.columns = df.columns.str.upper()

# Casos formatting
df["CASOS"] = df["CASOS"].astype(str)
df["CASOS"] = df["CASOS"].replace(',','',regex=True)
df["CASOS"] = df["CASOS"].astype(float)
#df["CASOS"] = df["CASOS"].fillna(0)
#df["CASOS"] = df["CASOS"].astype(int)

# Dataframe pre visualization
df

# Cases by district

In [ ]:
shape_1 = gpd.read_file(r'../../_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp')
shape_1['UBIGEO'] = shape_1['UBIGEO'].astype(str).astype(int)


# General cases by district 2021 data frame 
cases_dist_2021 = df.loc[df['AÑO'] == 2021]

# Cases by district
dist_21 = cases_dist_2021.groupby(['UBIGEO'])[['CASOS']].sum()
dist_21["UBIGEO"] = dist_21.index
dist_21.reset_index(drop=True, inplace=True)
dist_21

# Cases by district georeferenced
cases_dist = pd.merge(shape_1, dist_21, how="outer", on="UBIGEO")

# Plot
cases_dist.plot( column='CASOS', cmap='Reds', 
          figsize=(15, 15),
          linestyle='-',
          edgecolor='gray',
          legend = True,
          missing_kwds={"color": "lightgrey"})

# Cases by province

# Cases by department and year

Generating departamental data

In [ ]:
shape_3 = gpd.read_file(r'../../_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp')

# Generating Ubigeo as string, cause not all Ubigeo values are of 6 digits
shape_3["UBIGEO_STR"]  = shape_3['UBIGEO'].astype(str).str.zfill(6)

# Generate province ID
shape_3["UBIGEO_DEP"] = shape_3["UBIGEO_STR"].str[0:2].astype(int)
shape_3['UBIGEO'] = shape_3['UBIGEO'].astype(str).astype(int)

# Agregating shapes by department
shape_3_dep = shape_3.dissolve(by='UBIGEO_DEP')

# Generating departmental data
dep = df.groupby(['UBIGEO_DEP','AÑO'])[['CASOS']].sum()
dep["UBIGEO_DEP"] = dep.index.get_level_values(0)
dep["AÑO"] = dep.index.get_level_values(1)
dep.reset_index(drop=True, inplace=True)
dep.sort_values('AÑO')

Generating data frames by year

In [ ]:
year_list = [2015, 2016, 2017, 2018, 2019, 2020, 2021]


cases_dep = {}
dep_year = {}

for i in range(len(year_list)):
    cases_dep[year_list[i]] = dep.loc[dep['AÑO'] == year_list[i]]
    dep_year[year_list[i]] = pd.merge(shape_3_dep, cases_dep[year_list[i]], how="outer", on="UBIGEO_DEP")

Plotting data

In [ ]:
# A small function that hellps us for automatic subplots dimmensions
from math import sqrt
def nearest_square(num):
         num1 = round(sqrt(num))**2
         return int(sqrt(num1))

# Figure set up
fig, axis = plt.subplots( nrows = nearest_square(len(year_list)),
                         ncols= nearest_square(len(year_list)),
                         figsize = ( 15, 15 ) )

# Loop for subplot
idx = 0
for i in range( nearest_square(len(year_list)) ):
    for j in range ( nearest_square(len(year_list)) ):
        
        # if idx is minor to number of elements
        if idx <= len(year_list) - 1:
            ax = axis[ i ][ j ]

            dep_year[year_list[idx]].plot( column='CASOS', 
                      cmap='Reds', 
                      linestyle='-',
                      edgecolor='black',
                      legend = True, 
                      ax = ax,
                      missing_kwds={"color": "lightgrey"}
                    )

            ax.set_title( year_list[idx] )

            idx = idx + 1
            
        # if idx is major to number of elements        
        else:
        
            axis[i,j].axis('off')
            idx = idx + 1
            
fig.suptitle('Dengue Cases by Year')
fig.tight_layout()

# Cases by department and 2021 quarters